In [1]:
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import plotly.graph_objects as go
import datetime
import pandas as pd
import csv
import os
import time
from dateutil import tz
import glob
import quandl as q
import re
import plotly.graph_objects as go
import numpy as np
import streamlit as st
from plotly.subplots import make_subplots

st.set_page_config(layout="wide")
# %%
from fredapi import Fred

In [2]:
api_key_fred = os.environ['api_key_fred']
fred = Fred(api_key=api_key_fred)

quandl_api_key = os.environ['quandl_api_key']


In [3]:
# # # start - read in BTC data # # #
datasource_btcusd = "BITFINEX/DOGEUSD.csv"
# dogeusd_data = pd.read_csv("coindata/{}".format(
#     datasource_btcusd.replace("/", " ")), index_col=0)
# dogeusd_data.index = pd.to_datetime(dogeusd_data.index)

# most_recent_stored_btcusd_date = dogeusd_data.sort_index().tail(1).index[0].strftime("%Y-%m-%d")
todays_date = datetime.date.today() - timedelta(days=1)
todays_date = todays_date.strftime("%Y-%m-%d")

data = q.get(datasource_btcusd.split(".")[0],   start_date='2017-01-01',
                 end_date='{}'.format(todays_date),
                 api_key=quandl_api_key)
data.info()
data["First"] = data.Last.shift(1)
data = data.dropna()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 829 entries, 2021-04-21 to 2023-07-28
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   High    829 non-null    float64
 1   Low     829 non-null    float64
 2   Mid     829 non-null    float64
 3   Last    829 non-null    float64
 4   Bid     829 non-null    float64
 5   Ask     829 non-null    float64
 6   Volume  829 non-null    float64
dtypes: float64(7)
memory usage: 51.8 KB


,High,Low,Mid,Last,Bid,Ask,Volume,First
Date,,,,,,,,
2021-04-22,0.310190,0.254200,0.263865,0.266120,0.263390,0.264340,3.914806e+07,0.308760
2021-04-23,0.268080,0.179490,0.243185,0.243570,0.242950,0.243420,7.169683e+07,0.266120
2021-04-24,0.290000,0.228900,0.270640,0.271600,0.270350,0.270930,3.672039e+07,0.243570
2021-04-25,0.289400,0.225320,0.250750,0.250880,0.250350,0.251150,1.529839e+07,0.271600
2021-04-26,0.280660,0.248000,0.269230,0.269110,0.268930,0.269530,1.969569e+07,0.250880
...,...,...,...,...,...,...,...,...
2023-07-24,0.077904,0.070269,0.074858,0.074826,0.074848,0.074869,1.722498e+07,0.071713
2023-07-25,0.083827,0.073699,0.081423,0.081388,0.081399,0.081447,1.796901e+07,0.074826
2023-07-26,0.082366,0.076671,0.078001,0.078047,0.077986,0.078016,1.088540e+07,0.081388


In [4]:
dogeusd_data = data.sort_index()
# store current df with up-to-date values
dogeusd_data.to_csv('coindata/{}'.format(
    datasource_btcusd.replace("/", " ")), index=True)

In [5]:
# # # start - data processing # # #
dogeusd_data = dogeusd_data.dropna()
dogeusd_data["350_movingaverage"] = pd.Series.rolling(
    dogeusd_data["Last"], window=350, min_periods=1).mean()
dogeusd_data["111_movingaverage"] = pd.Series.rolling(
    dogeusd_data["Last"], window=111, min_periods=1).mean()
# # # end - data processing # # #


In [6]:
# run_it = st.sidebar.button('Show visualizations')

# TODO: Future integrate ETH
# display_name_all_twitter_user_scraped_csvs, all_twitter_user_scraped_csvs = get_all_stored_crypto_csvs()
# display_name_user_selection_list_containing_twitter_user = st.sidebar.selectbox(
#     "Select existing Twitter-User", list(display_name_all_twitter_user_scraped_csvs), 0)


In [7]:
# # # start - get data from fred api and store csv # # #
# # GET S&P DATA
# Billions of Dollars 
data_SP500 = fred.get_series('SP500')
data_SP500 = data_SP500.sort_index()
# store current df with up-to-date values
data_SP500.to_csv('coindata/data_SP500.csv', index=True)

# # Assets: Total Assets: Total Assets: Wednesday Level (RESPPANWW)
#Assets: Total Assets: Total Assets: Wednesday Level (RESPPANWW) Millions of Dollars
data_WALCL = fred.get_series('WALCL')
data_WALCL.dropna()
data_WALCL = data_WALCL.sort_index()
# store current df with up-to-date values
data_WALCL.to_csv('coindata/data_WALCL.csv', index=True)

# # Overnight Reverse Repurchase Agreements Treasury Securities Sold by the Federal Reserve in the Temporary Open Market Operations (RRPONTSYD)
data_RRPONTSYD = fred.get_series('RRPONTSYD')
#Assets: Total Assets: Total Assets: Wednesday Level (RESPPANWW)
# Billions of U.S. Dollars
# data_FRED_RRPONTSYD.values = data_FRED_RRPONTSYD.values * 1000 
data_RRPONTSYD = data_RRPONTSYD * 1000 
data_RRPONTSYD.dropna()
data_RRPONTSYD = data_RRPONTSYD.sort_index()
# store current df with up-to-date values
data_RRPONTSYD.to_csv('coindata/data_RRPONTSYD.csv', index=True)

# # Deposits with Federal Reserve Banks, other than Reserve Balances: U.S. Treasury, General Account (WTREGEN)
# Billions of Dollars 
data_FRED_WTREGEN = fred.get_series('WTREGEN')
data_FRED_WTREGEN = data_FRED_WTREGEN * 1000

data_FRED_WTREGEN.dropna()
data_FRED_WTREGEN = data_FRED_WTREGEN.sort_index()
# store current df with up-to-date values
data_FRED_WTREGEN.to_csv('coindata/data_FRED_WTREGEN.csv', index=True)

# # net liquidity vs s&p500 weekly - Diagram
datasource_fred_total_assets = "data_WALCL.csv"
fred_total_assets = pd.read_csv("coindata/{}".format(
    datasource_fred_total_assets.replace("/", " ")), index_col=0)
fred_total_assets.index = pd.to_datetime(fred_total_assets.index)

most_recent_stored_fred_rrpontsyd_date = fred_total_assets.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")
todays_date = datetime.date.today() - timedelta(days=1)
todays_date = todays_date.strftime("%Y-%m-%d")

datasource_FRED_WTREGEN = "data_FRED_WTREGEN.csv"
FRED_WTREGEN_data = pd.read_csv("coindata/{}".format(
    datasource_FRED_WTREGEN.replace("/", " ")), index_col=0)
FRED_WTREGEN_data.index = pd.to_datetime(FRED_WTREGEN_data.index)

most_recent_stored_FRED_WTREGEN_date = FRED_WTREGEN_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")

datasource_FRED_RRPONTSYD = "data_RRPONTSYD.csv"
FRED_RRPONTSYD_data = pd.read_csv("coindata/{}".format(
    datasource_FRED_RRPONTSYD.replace("/", " ")), index_col=0)
FRED_RRPONTSYD_data.index = pd.to_datetime(FRED_RRPONTSYD_data.index)

most_recent_stored_FRED_RRPONTSYD_date = FRED_RRPONTSYD_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")

# # # end - get data from fred api and store csv # # #

In [8]:
dogeusd_data = dogeusd_data[(dogeusd_data.index > '2020-08-11 00:00:00')]

In [9]:
import warnings
warnings.filterwarnings("ignore")
from astral import moon
import datetime

# take the data_SP500 index and iterate over it to get the moon phase for each day
# the make a new df out of it called df_moon_phase
df_moon_phase = pd.DataFrame()
for i in data_SP500.index:
    df_moon_phase = df_moon_phase.append({"date": i, "moon_phase": moon.phase(i)}, ignore_index=True)

df_moon_phase["moon_phase"] = df_moon_phase["moon_phase"].astype(int)
df_moon_phase["date"] = pd.to_datetime(df_moon_phase["date"])
df_moon_phase["date"] = df_moon_phase["date"].dt.date
df_moon_phase = df_moon_phase.set_index("date")

# check the relationship between moon phase and dogeusd_data
# merge sp500 with moon phase
dogeusd_data_and_moon_phase = pd.merge(
    dogeusd_data, df_moon_phase, left_index=True, right_index=True, how='left')
dogeusd_data_and_moon_phase = dogeusd_data_and_moon_phase.dropna()
dogeusd_data_and_moon_phase["350_movingaverage"] = pd.Series.rolling(
    dogeusd_data_and_moon_phase["Last"], window=350, min_periods=1).mean()
dogeusd_data_and_moon_phase["111_movingaverage"] = pd.Series.rolling(
    dogeusd_data_and_moon_phase["Last"], window=111, min_periods=1).mean()

# calculat the daily price change for dogeusd_data_and_moon_phase in percentage
dogeusd_data_and_moon_phase["daily_price_change"] = dogeusd_data_and_moon_phase["Last"].pct_change()
dogeusd_data_and_moon_phase["daily_price_change"] = dogeusd_data_and_moon_phase["daily_price_change"].fillna(0)
# calculat the daily price change for dogeusd_data_and_moon_phase in absolut values
dogeusd_data_and_moon_phase["daily_price_change_abs"] = dogeusd_data_and_moon_phase["Last"].diff()
dogeusd_data_and_moon_phase["daily_price_change_abs"] = dogeusd_data_and_moon_phase["daily_price_change_abs"].fillna(0)
dogeusd_data_and_moon_phase

,High,Low,Mid,Last,Bid,Ask,Volume,First,350_movingaverage,111_movingaverage,moon_phase,daily_price_change,daily_price_change_abs
Date,,,,,,,,,,,,,
2021-04-22,0.310190,0.254200,0.263865,0.266120,0.263390,0.264340,3.914806e+07,0.308760,0.266120,0.266120,9.0,0.000000,0.000000
2021-04-23,0.268080,0.179490,0.243185,0.243570,0.242950,0.243420,7.169683e+07,0.266120,0.254845,0.254845,10.0,-0.084736,-0.022550
2021-04-26,0.280660,0.248000,0.269230,0.269110,0.268930,0.269530,1.969569e+07,0.250880,0.259600,0.259600,13.0,0.104857,0.025540
2021-04-27,0.280080,0.265080,0.271455,0.271210,0.271230,0.271680,1.280031e+07,0.269110,0.262503,0.262503,14.0,0.007804,0.002100
2021-04-28,0.344710,0.255940,0.318290,0.318830,0.318070,0.318510,4.295062e+07,0.271210,0.273768,0.273768,15.0,0.175583,0.047620
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-24,0.077904,0.070269,0.074858,0.074826,0.074848,0.074869,1.722498e+07,0.071713,0.081794,0.074304,5.0,0.016230,0.001195
2023-07-25,0.083827,0.073699,0.081423,0.081388,0.081399,0.081447,1.796901e+07,0.074826,0.081675,0.074240,6.0,0.087697,0.006562
2023-07-26,0.082366,0.076671,0.078001,0.078047,0.077986,0.078016,1.088540e+07,0.081388,0.081528,0.074173,7.0,-0.041050,-0.003341


In [10]:
asdasd

NameError: name 'asdasd' is not defined

In [11]:
# now we want the relationship of the moon_phase column on the daily_price_change and daily_price_change_abs column
# we group by the moon_phase column and calculate the mean of the daily_price_change and daily_price_change_abs column
dogeusd_data_and_moon_phase_grouped = dogeusd_data_and_moon_phase.groupby("moon_phase").mean()
dogeusd_data_and_moon_phase_grouped = dogeusd_data_and_moon_phase_grouped.reset_index()
dogeusd_data_and_moon_phase_grouped


,moon_phase,High,Low,Mid,Last,Bid,Ask,Volume,First,350_movingaverage,111_movingaverage,daily_price_change,daily_price_change_abs
0,0.0,0.167800,0.150045,0.157203,0.157192,0.157128,0.157278,2.155073e+07,0.162338,0.227809,0.186009,-0.003364,-0.003345
1,1.0,0.140844,0.124019,0.135327,0.135267,0.135226,0.135428,2.117920e+07,0.133925,0.199749,0.156990,0.002893,0.001202
2,2.0,0.149344,0.134378,0.143363,0.143352,0.143295,0.143430,2.188213e+07,0.140102,0.206574,0.168366,0.002521,0.001845
3,3.0,0.145262,0.133185,0.138508,0.138537,0.138446,0.138569,2.452539e+07,0.136959,0.210777,0.167682,0.015103,0.001963
4,4.0,0.153300,0.141289,0.146592,0.146630,0.146525,0.146658,1.701858e+07,0.149049,0.216571,0.171548,-0.012722,-0.004704
5,5.0,0.150686,0.138979,0.143089,0.143092,0.143030,0.143148,1.496781e+07,0.146186,0.207544,0.169316,-0.016475,-0.003860
6,6.0,0.156861,0.135113,0.145198,0.145112,0.145107,0.145289,2.874561e+07,0.151227,0.221842,0.180437,0.019190,-0.003651
7,7.0,0.154174,0.137546,0.146774,0.146788,0.146707,0.146841,1.807228e+07,0.146435,0.214538,0.172473,-0.000026,0.002247
8,8.0,0.136241,0.121945,0.127560,0.127555,0.127498,0.127622,1.872995e+07,0.131200,0.187993,0.146694,-0.006634,-0.002895
9,9.0,0.143957,0.128904,0.134725,0.134829,0.134647,0.134804,1.969677e+07,0.139425,0.201343,0.163933,-0.015370,-0.001734


In [12]:
# # plot the moon_phase vs the daily_price_change as seperate lines with the time on the x-axis
# fig_moon_phase = make_subplots(specs=[[{"secondary_y": True}]])
# fig_moon_phase.add_trace(
#     go.Scatter(
#         x=dogeusd_data_and_moon_phase_grouped["moon_phase"],
#         y=dogeusd_data_and_moon_phase_grouped["daily_price_change"],
#         name="daily_price_change",
#         mode='lines',
#         marker=dict(color="red"),
#     )
# )

# fig_moon_phase

In [13]:
dogeusd_data_and_moon_phase["validate_ta_logic_category"] = ""
dogeusd_data_and_moon_phase.loc[(dogeusd_data_and_moon_phase["moon_phase"] > 11) | (dogeusd_data_and_moon_phase["moon_phase"] <= 3), "validate_ta_logic_category"] = "long"
dogeusd_data_and_moon_phase.loc[(dogeusd_data_and_moon_phase["moon_phase"] >3) & (dogeusd_data_and_moon_phase["moon_phase"] <=11), "validate_ta_logic_category"] = "short"
dogeusd_data_and_moon_phase

,High,Low,Mid,Last,Bid,Ask,Volume,First,350_movingaverage,111_movingaverage,moon_phase,daily_price_change,daily_price_change_abs,validate_ta_logic_category
Date,,,,,,,,,,,,,,
2021-04-22,0.310190,0.254200,0.263865,0.266120,0.263390,0.264340,3.914806e+07,0.308760,0.266120,0.266120,9.0,0.000000,0.000000,short
2021-04-23,0.268080,0.179490,0.243185,0.243570,0.242950,0.243420,7.169683e+07,0.266120,0.254845,0.254845,10.0,-0.084736,-0.022550,short
2021-04-26,0.280660,0.248000,0.269230,0.269110,0.268930,0.269530,1.969569e+07,0.250880,0.259600,0.259600,13.0,0.104857,0.025540,long
2021-04-27,0.280080,0.265080,0.271455,0.271210,0.271230,0.271680,1.280031e+07,0.269110,0.262503,0.262503,14.0,0.007804,0.002100,long
2021-04-28,0.344710,0.255940,0.318290,0.318830,0.318070,0.318510,4.295062e+07,0.271210,0.273768,0.273768,15.0,0.175583,0.047620,long
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-24,0.077904,0.070269,0.074858,0.074826,0.074848,0.074869,1.722498e+07,0.071713,0.081794,0.074304,5.0,0.016230,0.001195,short
2023-07-25,0.083827,0.073699,0.081423,0.081388,0.081399,0.081447,1.796901e+07,0.074826,0.081675,0.074240,6.0,0.087697,0.006562,short
2023-07-26,0.082366,0.076671,0.078001,0.078047,0.077986,0.078016,1.088540e+07,0.081388,0.081528,0.074173,7.0,-0.041050,-0.003341,short


In [14]:
# 0 .. 6.99	New moon
# 7 .. 13.99	First quarter
# 14 .. 20.99	Full moon
# 21 .. 27.99	Last quarter
# create a new column called moon_phase_category and fill it with the moon_phase_category
dogeusd_data_and_moon_phase["moon_phase_category"] = ""
dogeusd_data_and_moon_phase.loc[(dogeusd_data_and_moon_phase["moon_phase"] >= 0) & (dogeusd_data_and_moon_phase["moon_phase"] <= 6.99), "moon_phase_category"] = "New moon"
dogeusd_data_and_moon_phase.loc[(dogeusd_data_and_moon_phase["moon_phase"] >= 7) & (dogeusd_data_and_moon_phase["moon_phase"] <= 13.99), "moon_phase_category"] = "First quarter"
dogeusd_data_and_moon_phase.loc[(dogeusd_data_and_moon_phase["moon_phase"] >= 14) & (dogeusd_data_and_moon_phase["moon_phase"] <= 20.99), "moon_phase_category"] = "Full moon"
dogeusd_data_and_moon_phase.loc[(dogeusd_data_and_moon_phase["moon_phase"] >= 21) & (dogeusd_data_and_moon_phase["moon_phase"] <= 27.99), "moon_phase_category"] = "Last quarter"
dogeusd_data_and_moon_phase

,High,Low,Mid,Last,Bid,Ask,Volume,First,350_movingaverage,111_movingaverage,moon_phase,daily_price_change,daily_price_change_abs,validate_ta_logic_category,moon_phase_category
Date,,,,,,,,,,,,,,,
2021-04-22,0.310190,0.254200,0.263865,0.266120,0.263390,0.264340,3.914806e+07,0.308760,0.266120,0.266120,9.0,0.000000,0.000000,short,First quarter
2021-04-23,0.268080,0.179490,0.243185,0.243570,0.242950,0.243420,7.169683e+07,0.266120,0.254845,0.254845,10.0,-0.084736,-0.022550,short,First quarter
2021-04-26,0.280660,0.248000,0.269230,0.269110,0.268930,0.269530,1.969569e+07,0.250880,0.259600,0.259600,13.0,0.104857,0.025540,long,First quarter
2021-04-27,0.280080,0.265080,0.271455,0.271210,0.271230,0.271680,1.280031e+07,0.269110,0.262503,0.262503,14.0,0.007804,0.002100,long,Full moon
2021-04-28,0.344710,0.255940,0.318290,0.318830,0.318070,0.318510,4.295062e+07,0.271210,0.273768,0.273768,15.0,0.175583,0.047620,long,Full moon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-24,0.077904,0.070269,0.074858,0.074826,0.074848,0.074869,1.722498e+07,0.071713,0.081794,0.074304,5.0,0.016230,0.001195,short,New moon
2023-07-25,0.083827,0.073699,0.081423,0.081388,0.081399,0.081447,1.796901e+07,0.074826,0.081675,0.074240,6.0,0.087697,0.006562,short,New moon
2023-07-26,0.082366,0.076671,0.078001,0.078047,0.077986,0.078016,1.088540e+07,0.081388,0.081528,0.074173,7.0,-0.041050,-0.003341,short,First quarter


In [15]:
# group the dogeusd_data_and_moon_phase by the moon_phase_category column and calculate the mean of the daily_price_change and daily_price_change_abs column
dogeusd_data_and_moon_phase_grouped = dogeusd_data_and_moon_phase.groupby("moon_phase_category").mean()
dogeusd_data_and_moon_phase_grouped = dogeusd_data_and_moon_phase_grouped.reset_index()
dogeusd_data_and_moon_phase_grouped

,moon_phase_category,High,Low,Mid,Last,Bid,Ask,Volume,First,350_movingaverage,111_movingaverage,moon_phase,daily_price_change,daily_price_change_abs
0,First quarter,0.147942,0.131457,0.139339,0.139358,0.139272,0.139406,2.555244e+07,0.140168,0.210526,0.169645,9.933333,-0.003058,-0.000579
1,Full moon,0.146837,0.133102,0.140058,0.140058,0.139992,0.140124,1.971324e+07,0.139732,0.210325,0.168805,17.020408,0.001491,0.000793
2,Last quarter,0.161608,0.143686,0.153359,0.153378,0.153277,0.153442,1.832457e+07,0.152551,0.216252,0.175070,23.904762,0.001503,0.000043
3,New moon,0.152311,0.136966,0.144444,0.144431,0.144370,0.144518,2.147396e+07,0.145966,0.213325,0.171781,2.945946,0.001103,-0.001518


In [16]:
# # group the dogeusd_data_and_moon_phase by the moon_phase_category column and calculate the mean of the daily_price_change and daily_price_change_abs column
# dogeusd_data_and_moon_phase_grouped = dogeusd_data_and_moon_phase.groupby("validate_ta_logic_category").mean()
# dogeusd_data_and_moon_phase_grouped = dogeusd_data_and_moon_phase_grouped.reset_index()
# dogeusd_data_and_moon_phase_grouped
# validate_ta_logic_category	High	Low	Mid	Last	Bid	Ask	Volume	First	350_movingaverage	111_movingaverage	moon_phase	daily_price_change	daily_price_change_abs
# 0	long	0.154273	0.138621	0.146826	0.146832	0.146752	0.146901	1.979144e+07	0.146288	0.214739	0.173295	15.864286	0.001697	0.000425
# 1	short	0.149864	0.132988	0.140668	0.140678	0.140597	0.140739	2.557094e+07	0.143205	0.212704	0.170402	7.509091	-0.004078	-0.002268

In [17]:
# Correlation Analysis: First, it would be good to calculate the correlation coefficient between the moon_phase_category and the daily_price_change
# we use the pandas function corr() to calculate the correlation coefficient
dogeusd_data_and_moon_phase["daily_price_change"].corr(dogeusd_data_and_moon_phase["moon_phase"])

0.02479197823607795

In [18]:
# Regression Analysis: A linear regression model could be used to predict the BTC price change based on the moon phase. If the model is statistically significant (which you can check using a hypothesis test for the model parameters), this would suggest that there is a relationship. Moreover, the regression coefficients can be interpreted as the expected change in the BTC price for each phase of the moon.
# we use the statsmodels package to run the regression
import statsmodels.api as sm
# we need to add a constant term to the data set to estimate the intercept of the linear regression model
X = sm.add_constant(dogeusd_data_and_moon_phase["moon_phase"])
# we run the regression
model = sm.OLS(dogeusd_data_and_moon_phase["daily_price_change"], X)
results = model.fit()
# we print the summary of the regression
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:     daily_price_change   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.3629
Date:                Sat, 29 Jul 2023   Prob (F-statistic):              0.547
Time:                        13:51:29   Log-Likelihood:                 752.11
No. Observations:                 592   AIC:                            -1500.
Df Residuals:                     590   BIC:                            -1491.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0026      0.005     -0.471      0.6

In [19]:
#                          OLS Regression Results                              
# ==================================================================================
# Dep. Variable:     daily_price_change_abs   R-squared:                       0.007
# Model:                                OLS   Adj. R-squared:                  0.006
# Method:                     Least Squares   F-statistic:                     5.527
# Date:                    Thu, 20 Jul 2023   Prob (F-statistic):             0.0190
# Time:                            16:08:07   Log-Likelihood:                -6671.4
# No. Observations:                     765   AIC:                         1.335e+04
# Df Residuals:                         763   BIC:                         1.336e+04
# Df Model:                               1                                         
# Covariance Type:                nonrobust                                         
# ==============================================================================
#                  coef    std err          t      P>|t|      [0.025      0.975]
# ------------------------------------------------------------------------------
# const       -186.6173    104.328     -1.789      0.074    -391.421      18.186
# moon_phase    15.6265      6.647      2.351      0.019       2.578      28.675
# ==============================================================================
# Omnibus:                       77.255   Durbin-Watson:                   1.994
# Prob(Omnibus):                  0.000   Jarque-Bera (JB):              478.758
# Skew:                          -0.151   Prob(JB):                    1.09e-104
# Kurtosis:                       6.864   Cond. No.                         30.6
# ==============================================================================

# Notes:
# [1] Standard Errors assume that the covariance matrix of the errors is correctly specified.

# Your regression output indicates that the moon phase variable is significant at the 5% level (since the p-value, denoted as "P>|t|" for "moon_phase",
# is less than 0.05). This means there is evidence to suggest that the moon phase has some influence on the absolute daily price change of Bitcoin.

# However, be careful in interpreting the magnitude and meaning of this relationship. The coefficient for "moon_phase" is 15.6265, 
# which means that for each unit increase in moon phase, the model predicts an average increase of 15.6265 units in the absolute daily price change of Bitcoin. 
# But because moon phase is probably a categorical variable coded as numbers, it might not make much sense to think about "unit increases" in this way.

# Moreover, the R-squared value of the model is quite low (0.007), meaning that the moon phase only explains a very small portion of the variation 
# in the absolute daily price change of Bitcoin. This suggests that there are many other factors influencing Bitcoin's daily price change, 
# and that the moon phase alone is not a strong predictor.

# Also, it's important to note that this is a simple linear regression analysis, and it doesn't account for potential confounding variables or 
# other complexities of financial time-series data, such as autocorrelation or non-stationarity.

# It's also worth mentioning again that correlation does not imply causation. Just because there is a statistically significant 
# relationship between moon phase and Bitcoin's price change, it doesn't mean the moon phase is causing these price changes.
# It could be due to chance or other variables that happen to correlate with the moon phase.

In [20]:
dogeusd_data_and_moon_phase.head()

,High,Low,Mid,Last,Bid,Ask,Volume,First,350_movingaverage,111_movingaverage,moon_phase,daily_price_change,daily_price_change_abs,validate_ta_logic_category,moon_phase_category
Date,,,,,,,,,,,,,,,
2021-04-22,0.31019,0.25420,0.263865,0.26612,0.26339,0.26434,3.914806e+07,0.30876,0.266120,0.266120,9.0,0.000000,0.00000,short,First quarter
2021-04-23,0.26808,0.17949,0.243185,0.24357,0.24295,0.24342,7.169683e+07,0.26612,0.254845,0.254845,10.0,-0.084736,-0.02255,short,First quarter
2021-04-26,0.28066,0.24800,0.269230,0.26911,0.26893,0.26953,1.969569e+07,0.25088,0.259600,0.259600,13.0,0.104857,0.02554,long,First quarter
2021-04-27,0.28008,0.26508,0.271455,0.27121,0.27123,0.27168,1.280031e+07,0.26911,0.262503,0.262503,14.0,0.007804,0.00210,long,Full moon
2021-04-28,0.34471,0.25594,0.318290,0.31883,0.31807,0.31851,4.295062e+07,0.27121,0.273768,0.273768,15.0,0.175583,0.04762,long,Full moon


In [21]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Normalizing the 'Volume'
dogeusd_data_and_moon_phase['norm_volume'] = (dogeusd_data_and_moon_phase['Volume'] - dogeusd_data_and_moon_phase['Volume'].min()) / (dogeusd_data_and_moon_phase['Volume'].max() - dogeusd_data_and_moon_phase['Volume'].min())

# Calculate daily price change
dogeusd_data_and_moon_phase['daily_change'] = dogeusd_data_and_moon_phase['Last'].diff()

# Assign colors based on positive or negative change
dogeusd_data_and_moon_phase['color'] = np.where(dogeusd_data_and_moon_phase['daily_change'] > 0, 'green', 'red')

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(
        x=dogeusd_data_and_moon_phase.index,
        y=dogeusd_data_and_moon_phase['Last'],
        mode='markers+lines',
        marker=dict(
            size=dogeusd_data_and_moon_phase['norm_volume']*10,  # Scale marker size
            color=dogeusd_data_and_moon_phase['color']  # Assign color based on 'daily_change'
        ),
        name='Last',
        line=dict(color='black')
    ),
    secondary_y=False
)

fig.add_trace(
    go.Bar(
        x=dogeusd_data_and_moon_phase.index,
        y=dogeusd_data_and_moon_phase['Volume'],
        name='Volume',
        marker=dict(color='black')
    ),
    secondary_y=True
)

# Add figure title and labels
fig.update_layout(
    title_text="DOGEUSD Last and Volume",
    xaxis_title="Date",
    yaxis_title="Last",
    yaxis2_title="Volume",
    xaxis_rangeslider_visible=True,
)

# Show the plot
fig.show()


In [22]:
import datetime
import requests

In [23]:
def get_binance_btcusd():
    URL = "https://api.binance.com/api/v3/klines"
    start_str = '2014-01-01 00:00:00'
    fmt = '%Y-%m-%d %H:%M:%S'
    start_time = int(time.mktime(time.strptime(start_str, fmt)) * 1000)
    last_open_time = 0  # added this line

    df = pd.DataFrame()

    while True:
        parameters = {
            'symbol': 'BTCUSDT',
            'interval': '1d',
            'startTime': start_time,
            'limit': 1000  # maximum limit
        }

        response = requests.get(URL, params=parameters)
        data = json.loads(response.text)
        
        if len(data) == 0 or last_open_time == start_time:  # updated this line
            break

        temp_df = pd.DataFrame(data, columns=['Open_time', 'Open', 'High', 'Low', 'Close', 'Volume', 
                                              'Close_time', 'Quote_asset_volume', 'Number_of_trades', 
                                              'Taker_buy_base', 'Taker_buy_quote', 'Ignore'])
        temp_df['Open_time'] = pd.to_datetime(temp_df['Open_time'], unit='ms')
        temp_df['Date'] = temp_df['Open_time'].dt.date
        temp_df['High'] = temp_df['High'].astype(float)
        temp_df['Low'] = temp_df['Low'].astype(float)
        temp_df['Last'] = temp_df['Close'].astype(float)
        temp_df['Volume'] = temp_df['Volume'].astype(float)
        temp_df['Mid'] = (temp_df['High'] + temp_df['Low']) / 2
        temp_df['First'] = temp_df['Last'].shift()

        df = pd.concat([df, temp_df])
        last_open_time = start_time  # added this line
        start_time = int(temp_df['Open_time'].dt.to_pydatetime()[-1].timestamp() * 1000) + 1

    df = df[['Date', 'High', 'Low', 'Mid', 'Last', 'Volume', 'First']]
    df.to_csv('coindata/binance_btcusdt.csv', index=False)

datasource_btcusd = "binance_btcusdt.csv"
btcusd_data = pd.read_csv("coindata/{}".format(datasource_btcusd), index_col=0)
btcusd_data.index = pd.to_datetime(btcusd_data.index)

most_recent_stored_btcusd_date = (
    btcusd_data.sort_index().tail(1).index[0].strftime("%Y-%m-%d")
)

todays_date = datetime.date.today()
todays_date = todays_date.strftime("%Y-%m-%d")


# if most_recent_stored_btcusd_date != todays_date:
# get_binance_btcusd()

# create moon diagrams

In [24]:
# 0 .. 6.99	New moon
# 7 .. 13.99	First quarter
# 14 .. 20.99	Full moon
# 21 .. 27.99	Last quarter
# create a new column called moon_phase_category and fill it with the moon_phase_category
dogeusd_data_and_moon_phase["moon_phase_category"] = ""
dogeusd_data_and_moon_phase.loc[(dogeusd_data_and_moon_phase["moon_phase"] >= 0) & (dogeusd_data_and_moon_phase["moon_phase"] <= 6.99), "moon_phase_category"] = 0
dogeusd_data_and_moon_phase.loc[(dogeusd_data_and_moon_phase["moon_phase"] >= 7) & (dogeusd_data_and_moon_phase["moon_phase"] <= 13.99), "moon_phase_category"] = 7
dogeusd_data_and_moon_phase.loc[(dogeusd_data_and_moon_phase["moon_phase"] >= 14) & (dogeusd_data_and_moon_phase["moon_phase"] <= 20.99), "moon_phase_category"] = 14
dogeusd_data_and_moon_phase.loc[(dogeusd_data_and_moon_phase["moon_phase"] >= 21) & (dogeusd_data_and_moon_phase["moon_phase"] <= 27.99), "moon_phase_category"] = 21


In [31]:
# make a new plot and plot the daily_price_change of btc by time on the x-axis
fig_btc_price_change = make_subplots(specs=[[{"secondary_y": True}]])
fig_btc_price_change.add_trace(
    go.Scatter(
        x=dogeusd_data_and_moon_phase.index,
        y=dogeusd_data_and_moon_phase["Last"],
        name="Last",
        mode='lines',
        marker=dict(color="red"),
    )
)

# now add the moon_phase column as a second y-axis
fig_btc_price_change.add_trace(
    go.Scatter(
        x=dogeusd_data_and_moon_phase.index,
        y=dogeusd_data_and_moon_phase["moon_phase_category"],
        name="moon_phase",
        mode='lines',
        marker=dict(color="blue"),
    ),
    secondary_y=True,
)

last_moon_phase = None

for i in range(len(dogeusd_data_and_moon_phase)):
    current_moon_phase = dogeusd_data_and_moon_phase['moon_phase_category'].iloc[i]
    if current_moon_phase != last_moon_phase:  # check if the moon phase category changed
        if current_moon_phase == 0:  
            fig_btc_price_change.add_annotation(
                x=dogeusd_data_and_moon_phase.index[i],
                y=0.9,
                text="🌑",
                showarrow=False,
                font=dict(
                    size=16,
                ),
                xref="x",
                yref="paper",  # Position annotation relative to the entire plot
                yanchor="bottom",  # Anchor the bottom of the text at y
            )
        elif current_moon_phase == 14: 
            fig_btc_price_change.add_annotation(
                x=dogeusd_data_and_moon_phase.index[i],
                y=0,
                text="🌕",
                showarrow=False,
                font=dict(
                    size=16,
                ),
                xref="x",
                yref="paper",  # Position annotation relative to the entire plot
                yanchor="bottom",  # Anchor the bottom of the text at y
            )
        last_moon_phase = current_moon_phase  # update the last moon phase

fig_btc_price_change

In [32]:
dogeusd_data_and_moon_phase.tail(6)

,High,Low,Mid,Last,Bid,Ask,Volume,First,350_movingaverage,111_movingaverage,moon_phase,daily_price_change,daily_price_change_abs,validate_ta_logic_category,moon_phase_category,norm_volume,daily_change,color
Date,,,,,,,,,,,,,,,,,,
2023-07-21,0.076865,0.070365,0.073760,0.073631,0.073725,0.073794,7.946412e+06,0.070563,0.081921,0.074423,3.0,0.043479,0.003068,long,0,0.017993,0.003068,green
2023-07-24,0.077904,0.070269,0.074858,0.074826,0.074848,0.074869,1.722498e+07,0.071713,0.081794,0.074304,5.0,0.016230,0.001195,short,0,0.041081,0.001195,green
2023-07-25,0.083827,0.073699,0.081423,0.081388,0.081399,0.081447,1.796901e+07,0.074826,0.081675,0.074240,6.0,0.087697,0.006562,short,0,0.042933,0.006562,green
2023-07-26,0.082366,0.076671,0.078001,0.078047,0.077986,0.078016,1.088540e+07,0.081388,0.081528,0.074173,7.0,-0.041050,-0.003341,short,7,0.025306,-0.003341,red
2023-07-27,0.080050,0.076774,0.077392,0.077365,0.077369,0.077414,4.555882e+06,0.078047,0.081360,0.074101,8.0,-0.008738,-0.000682,short,7,0.009556,-0.000682,red
2023-07-28,0.077977,0.076128,0.077591,0.077560,0.077570,0.077612,4.153710e+06,0.077365,0.081208,0.074039,9.0,0.002521,0.000195,short,7,0.008555,0.000195,green
